In [21]:
import plotly.express as px
from Project.Database import Db
from Project._07OptimiseConsumption.emission_reduction import create_gantt_data_and_event_dataframes, emission_reduction

In [5]:
movable_appliances = ['Load_StatusApplianceDishwasher', 'Load_StatusPlugLoadVacuum', 'Load_StatusClothesWasher',
                          'Load_StatusDryerPowerTotal', 'Load_StatusPlugLoadIron']

In [6]:
%%time
optim_year2 = emission_reduction(year=2)[0]

In [7]:
appliance_job_list, uo_event_df, o_event_df = create_gantt_data_and_event_dataframes(optim=optim_year2)

In [22]:
fig = px.timeline(appliance_job_list,
                  x_start="start",
                  x_end="end",
                  y="appliance",
                  color='CO2 Emission (kg)',
                  text='CO2 Emission (kg)',
                  category_orders={
                      "appliance": ['Dishwasher after',
                                    'Dishwasher before',
                                    'Clothes Washer after',
                                    'Clothes Washer before',
                                    'Dryer after',
                                    'Dryer before',
                                    'Iron after',
                                    'Iron before',
                                    'Vacuum after',
                                    'Vacuum before']})
fig.update_layout(template='plotly')
fig.write_html(Db.get_save_file_directory(f"Gantt/Gantt_of_optimisation.html"))

CPU times: user 127 ms, sys: 21.1 ms, total: 148 ms
Wall time: 153 ms


In [9]:
def compute_optimised_reduction(uo_event_df, o_event_df):
    event_difference = (o_event_df.groupby('Day').size() - uo_event_df.groupby('Day').size() > 0).loc[lambda self: self]
    uo_event_df.sum(), o_event_df.sum(), o_event_df.loc[
        lambda self: self['Day'].isin(event_difference.index.tolist())].groupby(
        'Day').max().sum()
    unoptimised_emission = uo_event_df['CO2 Emission (kg)'].sum()
    optimised_emission = o_event_df['CO2 Emission (kg)'].sum()
    real_optimised_emission = (o_event_df['CO2 Emission (kg)'].sum() - o_event_df.loc[
        lambda self: self['Day'].isin(event_difference.index.tolist())].groupby('Day').max().sum()).values[0]

    print('Unoptimised emission value:', unoptimised_emission,
          '\nFalse optimised emission:  ', optimised_emission,
          '\nTrue Optimised emission value:  ', round(real_optimised_emission, 2),
          '\nEvent difference:\n', event_difference)
    return round(unoptimised_emission - real_optimised_emission, 2)

In [10]:
emission_reduction_dict = {}
emission_reduction_dict[f'Year2'] = compute_optimised_reduction(uo_event_df, o_event_df)

Unoptimised emission value: 301.49 
False optimised emission:   320.85 
True Optimised emission value:   268.14 
Event difference:
 Day
3      True
10     True
17     True
24     True
31     True
38     True
45     True
52     True
59     True
66     True
73     True
80     True
87     True
94     True
101    True
108    True
115    True
122    True
129    True
136    True
143    True
150    True
157    True
164    True
171    True
178    True
185    True
192    True
199    True
206    True
213    True
220    True
234    True
241    True
248    True
255    True
269    True
276    True
283    True
290    True
297    True
304    True
311    True
318    True
325    True
332    True
339    True
346    True
353    True
360    True
dtype: bool


In [11]:
optim_year1 = emission_reduction(year=1)[0]

In [12]:
emission_reduction_dict[f'Year1'] = compute_optimised_reduction(*create_gantt_data_and_event_dataframes(optim=optim_year1)[1:])

Unoptimised emission value: 447.91999999999996 
False optimised emission:   448.65999999999997 
True Optimised emission value:   375.11 
Event difference:
 Day
16     True
23     True
30     True
37     True
44     True
51     True
58     True
65     True
72     True
79     True
86     True
93     True
100    True
107    True
114    True
121    True
128    True
135    True
142    True
149    True
156    True
163    True
170    True
177    True
184    True
191    True
198    True
205    True
212    True
219    True
226    True
233    True
240    True
247    True
254    True
261    True
268    True
275    True
282    True
289    True
296    True
303    True
310    True
317    True
324    True
331    True
338    True
345    True
352    True
359    True
dtype: bool


In [13]:
emission_reduction_dict

{'Year2': 33.35, 'Year1': 72.81}

In [17]:
emmision_reduction_year1 = optim_year1['m.a.u.o'][[f'{appliance}Emission' for appliance in movable_appliances if 'Dryer' not in appliance]].sum()-optim_year1['m.a.o'][[f'{appliance}Emission' for appliance in movable_appliances if 'Dryer' not in appliance]].sum()

emmision_reduction_year2 = optim_year2['m.a.u.o'][[f'{appliance}Emission' for appliance in movable_appliances if 'Dryer' not in appliance]].sum()-optim_year2['m.a.o'][[f'{appliance}Emission' for appliance in movable_appliances if 'Dryer' not in appliance]].sum()

In [18]:
emmision_reduction_year1['Dryer'] = emission_reduction_dict['Year1']
emmision_reduction_year2['Dryer'] = emission_reduction_dict['Year2']

In [19]:
emmision_reduction_year1.sum(), emmision_reduction_year2.sum()

(82.45953076957892, 46.76056941574669)